In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.ensemble import IsolationForest

In [ ]:
class AuctionAnalyzer:
    
    def __init__(self,client_id,client_secret): # init DataGrabr object, store id and secret
        self.id = client_id
        self.secret = client_secret
        
    def create_token(self, region = 'us'): #creates access token
        if hasattr(self, 'token') and 'expires_in' in self.token:
            # Check if token is still valid
            print(f"""Token already exists and is: {self.token['access_token']}, 
                  it expires in {(self.token['expiration_time'] - time.time())/ 3600} hours""")
            return 
            
        else:
            data = { 'grant_type': 'client_credentials' }
            response = requests.post('https://%s.battle.net/oauth/token' % region, data=data, auth=(self.id, self.secret))
            self.token = response.json()
            self.token['expiration_time'] = time.time() + 3600*24
            print(f"Your access token is: {self.token['access_token']}, it expires in {self.token['expires_in']/3600} hours")
        return self.token['access_token']
    
    def basic_auction_data(self,data,amount):
        ID_dict = {}
        for auction_listing in data['auctions'][:amount]:

            item_id = auction_listing['item']['id']
            price = auction_listing['buyout'] / 10000  # Convert to gold
            quantity = auction_listing['quantity']

            if item_id in ID_dict:
                ID_dict[item_id]['Count'].append(quantity)
                ID_dict[item_id]['Price Per Unit (in Gold)'].append(price)
            else:
                ID_dict[item_id] = {'Count':[quantity],'Price Per Unit (in Gold)':[price]}

        for i in ID_dict:
            ID_dict[i]['Count'] = sum(ID_dict[i]['Count'])
            ID_dict[i]['Market size (in Gold)'] = sum(ID_dict[i]['Price Per Unit (in Gold)'])
            ID_dict[i]['Avg Price (in Gold)'] = sum(ID_dict[i]['Price Per Unit (in Gold)']) / ID_dict[i]['Count']
            ID_dict[i]['Abs Min Overall'] = min(ID_dict[i]['Price Per Unit (in Gold)'])
        df = pd.DataFrame(ID_dict)

        return df.T
    
    
    def get_auctions(self, connected_realm_id = 60, region='us', n_listings = 90): 
        # Gets auction data from Stormrage by default
        '''Can control number of listings grabbed by adjusting parameter n_listings number--Gets 90 by default.
        By default the connected_realm_id is 60 which corresponds to the Stormrage Realm in region='us' .'''
        
        if not hasattr(self,"token") or (self.token['expiration_time'] - time.time()) < 0:
            raise AttributeError("Token DNE or is expired. Please request a new token.")
            
        else:
            self.n_listings = n_listings
            url = f'https://{region}.api.blizzard.com/data/wow/connected-realm/{connected_realm_id}/auctions'
            headers = {'Authorization': f"Bearer {self.token['access_token']}"}
            params = {'namespace': f"dynamic-{region}", 'locale': 'en_US'}
            response = requests.get(url, headers=headers, params=params)
            self.raw_auction = response.json()
            self.df_auction = self.basic_auction_data(self.raw_auction,self.n_listings)
            return self.df_auction
    
    def get_itemids(self):
        
        if not hasattr(self,"token") or (self.token['expiration_time'] - time.time()) < 0:
            raise AttributeError("Token DNE or is expired. Please request a new token.")
            
        if not hasattr(self,"df_auction"):
            raise AttributeError("Please run the get_auctions function first before requesting ItemIDs...")
            
        if not hasattr(self, "item_storage"):
            self.item_storage = {}
            
        max_requests_per_second = 100

        for i,ItemID in enumerate(self.df_auction.index):


            if hasattr(self, "item_storage") and ItemID in self.item_storage:
                self.df_auction.rename(index ={ItemID:self.item_storage[ItemID]}, inplace = True)

            else:       
                url = f'https://us.api.blizzard.com/data/wow/item/{ItemID}?namespace=static-us&locale=en_US'
                headers = {'Authorization': f"Bearer {self.token['access_token']}"}
                params  = {'namespace':'static-us','locale':'en_US'}
                response = requests.get(url, headers=headers, params=params)
                self.item_storage[ItemID] = response.json()['name']
                self.df_auction.rename(index ={ItemID:self.item_storage[ItemID]}, inplace = True)
                if i % 15 == 0:
                    print('Looting items...')
                # Check if we need to throttle
                if (i + 1) % max_requests_per_second == 0:
                    print("Throttling for 1 second to avoid exceeding rate limit...")
                    time.sleep(1)  # Wait for 1 second
        return 
    
    def update_items(self):
        count = 0
        not_found = 0
        if not hasattr(self,"df_auction"):
            raise AttributeError("Please run the get_auctions function first before attempting to update...")
            
        else:
            for i,ItemID in enumerate(self.df_auction.index):
                if type(ItemID) == str:
                    pass
                elif ItemID in self.item_storage:
                    self.df_auction.rename(index ={ItemID:self.item_storage[ItemID]}, inplace = True)
                    count += 1
                else:
                    not_found += 1
            print(f"""Successfully updated {count} items in 
                the auction house data from existing item storage.
                Failed to find {not_found} items. Run the get_itemids function to fix.""")
            
    def trim_outliers(self,mode='Limited',cap=20,random_state=0,contamination='auto',sel=[]):
        '''Uses the Isolation Forest Algorithm to identify and remove outliers from the data. Right now, I suggest looking
        for items that have a count higher than 20 for the Isolation Forest model.'''
    
        if not hasattr(self,"token") or (self.token['expiration_time'] - time.time()) < 0:
            raise AttributeError("Token DNE or is expired. Please request a new token.")

        if not hasattr(self,"df_auction"):
            raise AttributeError("Please run the get_auctions function first before attempting to filter data...")

        if not hasattr(self, "item_storage"):
            raise AttributeError("Please run the get_itemids function first before attempting to filter data...")

        else:
            if mode == 'Limited': # limits items to fit
                df_capped = self.df_auction[self.df_auction['Count'] > 20]
                df = df_capped.head(min(len(df_capped.index),cap))

            elif mode == 'Full': # DO NOT RUN YET,  need to do more testing
                pass
            
            elif mode == 'Selected':
                # fits for specific items
                df = self.df_auction.loc[sel]

            A = 'Adj Count'
            B = 'Adj Price Per Unit (in Gold)'
            C = 'Adj Market size (in Gold)'
            D = 'Adj Avg Price (in Gold)'
            E = 'Abs Min Overall' 
            F = 'Bottom 10'
            dic_storage = {}

            for Item in df.index:
                prices = df.loc[Item, 'Price Per Unit (in Gold)']
                X = np.array(prices).reshape(-1,1)
                Y = []
                min_prices = df.loc[Item,E]  #keeps min
                # We will keep the bottom 10 prices for the item 
                # If there is a significant difference between the average of the bottom few items and adj avg
                # more analysis will be needed
                bottom_10 = sorted(prices)[:10]
                
                #debugging
                #print(f"\nProcessing Item: {Item}")
                #print(f"Raw prices type: {type(prices)}")
                #print(f"Raw prices: {prices}")
                
                # fit model
                iso_forest  = IsolationForest(random_state=random_state, contamination=contamination)
                iso_forest.fit(X)
                outliers = iso_forest.predict(X)
                filtered_prices = [X[i][0] for i in range(len(X)) if outliers[i] == 1]
                
                if len(filtered_prices) == 0:
                    print(f"Item {Item} has no valid data for isolation forest."
                          "This sometimes happens due to contamination being too aggressive"
                          "Try setting it around 0.1-0.2")
                    continue
                    
                count = len(filtered_prices)
                total = sum(filtered_prices)
                average = total / count
                
                #debugging 
                #print(f"Shaped X: {X.shape}, Filter {filtered_prices}, outlier model {outliers}")
                #print(f"X values: {X}")
        
                
                dic_storage[Item] = {
                A: count,
                B: filtered_prices,
                C: total,
                D: average,
                E: min_prices,
                F: bottom_10}

            self.filtered_auction = pd.DataFrame(dic_storage).T
            return self.filtered_auction